In deze opgave classificeren afbeeldingen van bloemen uit de iris dataset met behulp van een neuraal netwerk.

In [327]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Laad de dataset en maak een feature matrix X en een target vector y.

In [328]:
iris = sns.load_dataset('iris')

# we gebruiken .values om een Numpy array te krijgen in plaats van een Pandas DataFrame
X_iris = iris.drop('species', axis=1).values 
y_iris = iris['species'].values

In [329]:
y_iris = y_iris.reshape(-1, 1) # maak een array van array's, dit is nodig voor de volgende stap
print(y_iris[:10])

[['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']
 ['setosa']]


Een neuraal netwerk kan niet omgaan met categorische labels zoals de namen van de planten, maar heeft getallen als uitput nodig. We hebben hier drie klassen (setosa, versicolor, virginica) daarom maken we een target vector met per label drie waarden. Een 1 voor de eerste waarde correspondeert met 'setosa', een 1 voor de 2e waade met 'versicolor' en een 1 voor de derde waarde met 'virginica'.

Doe dit met behulp van het `OneHotEncoder` object uit sklearn.

In [330]:
# Stap 1: Maak een instantie van de OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

# Stap 2: Pas de OneHotEncoder toe op de categorische kolom ('species')
encoded_data = enc.fit_transform(y_iris)

# Stap 3: Converteer de geëncodeerde data naar een DataFrame en voeg het toe aan de oorspronkelijke dataset
enc_df = pd.DataFrame(encoded_data.toarray(), columns=enc.get_feature_names_out(['species']))
iris_encoded = pd.concat([iris, enc_df], axis=1)

# Stap 4: Bekijk het resulterende DataFrame
print(iris_encoded)

     sepal_length  sepal_width  petal_length  petal_width    species  \
0             5.1          3.5           1.4          0.2     setosa   
1             4.9          3.0           1.4          0.2     setosa   
2             4.7          3.2           1.3          0.2     setosa   
3             4.6          3.1           1.5          0.2     setosa   
4             5.0          3.6           1.4          0.2     setosa   
..            ...          ...           ...          ...        ...   
145           6.7          3.0           5.2          2.3  virginica   
146           6.3          2.5           5.0          1.9  virginica   
147           6.5          3.0           5.2          2.0  virginica   
148           6.2          3.4           5.4          2.3  virginica   
149           5.9          3.0           5.1          1.8  virginica   

     species_setosa  species_versicolor  species_virginica  
0               1.0                 0.0                0.0  
1            

We hebben de data nu in een geschikt formaat. Splits de data in training en test data.

In [331]:
X_train, X_test, y_train, y_test = train_test_split(X_iris, enc_df, test_size=0.2, random_state=42)

Maak een eenvoudig neuraal netwerk met 3 lagen, waarin elke neuron in een laag met elk neuron in de vorige laag verbonden is ('Dense'). 
Voeg voor de eeste laag een parameter `input_shape` toe die aangeeft hoeveel features een element in 'X' heeft.
Het aantal neuronen in de laaste laag is gelijk aan het aantal waarden in een label in `y`. 

In [332]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

#voeg lagen toe met model.add()
model.add(Dense(8, input_shape = (4,), activation = 'relu'))
model.add(Dense(6, activation = 'tanh'))
model.add(Dense(3, activation = 'sigmoid'))


Compileer en bekijk het netwerk door onderstaande code te runnen.

In [333]:
from keras.optimizers import Adam

model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_122 (Dense)           (None, 8)                 40        
                                                                 
 dense_123 (Dense)           (None, 6)                 54        
                                                                 
 dense_124 (Dense)           (None, 3)                 21        
                                                                 
Total params: 115 (460.00 Byte)
Trainable params: 115 (460.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Nu kunnen we ons model trainen met behulp van de `fit` methode.

In [334]:
model.fit(X_train, y_train, epochs = 25, batch_size = 5)

Epoch 1/25


24/24 [==============================] - 1s 3ms/step - loss: 1.5367 - accuracy: 0.3417
Epoch 2/25
24/24 [==============================] - 0s 3ms/step - loss: 1.2291 - accuracy: 0.4583
Epoch 3/25
24/24 [==============================] - 0s 3ms/step - loss: 1.1199 - accuracy: 0.4000
Epoch 4/25
24/24 [==============================] - 0s 3ms/step - loss: 0.9928 - accuracy: 0.5917
Epoch 5/25
24/24 [==============================] - 0s 3ms/step - loss: 0.8648 - accuracy: 0.8917
Epoch 6/25
24/24 [==============================] - 0s 3ms/step - loss: 0.8015 - accuracy: 0.9083
Epoch 7/25
24/24 [==============================] - 0s 3ms/step - loss: 0.7634 - accuracy: 0.9167
Epoch 8/25
24/24 [==============================] - 0s 3ms/step - loss: 0.7309 - accuracy: 0.9417
Epoch 9/25
24/24 [==============================] - 0s 3ms/step - loss: 0.7041 - accuracy: 0.9083
Epoch 10/25
24/24 [==============================] - 0s 3ms/step - loss: 0.6831 - accuracy: 0.9083
Epoch 11/25
24/24 [===========

Met `model_evaluate` kunnen we bepalen hoe goed het model werkt op de test data.

In [335]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)

1/1 [==============================] - 0s 384ms/step - loss: 0.4290 - accuracy: 0.9667
0.9666666388511658


Experimenteer met bovenstaand model. Wat gebeurt er als je meer lagen toevoegt of een laag weghaalt of het aantal neuronen in een laag verandert? 

Probeer ook eens andere activatiefuncties dan `relu`, zoals `tanh` of `sigmoid`.

Welke invloed heeft het aantal epochs?

Het aantal epochs heeft invloed op hoe lang het model wordt getraind en hoe goed het leert van de data. Een te laag aantal epochs kan resulteren in een slecht getraind model dat niet goed generaliseert naar nieuwe data, terwijl een te hoog aantal epochs kan leiden tot overfitting. 

We kunnen dezelfde aanpak gebruiken voor andere data, bijvoorbeeld voor het herkennen van cijfers. We gebruiken de `MNist` dataset die bestaat uit plaatjes van cijfers. Ieder plaatje bestaat uit 28 x 28 pixels.

In [336]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

import warnings
warnings.filterwarnings('ignore')

In [337]:
train_images.shape

(60000, 28, 28)

In [338]:
len(train_labels)

60000

In [339]:
print(train_labels)

[5 0 4 ... 5 6 8]


Een eenvoudig neuraal netwerk heeft een array van inputwaarden nodig i.p.v. een 2D afbeelding.
We maken van de inputs (waarden tussen 0 en 255) getallen tussen 0 en 1.

In [340]:
train_images = train_images.reshape((60000, 28 * 28)) # lijst van waarden i.p.v. 2D afbeelding
train_images = train_images.astype('float32') / 255 # getallen tussen 0 en 1

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Voor elk label (een cijfer) maken we een array met 10 waarden: 9 nullen en een één, waarbij de positie van de één aangeeft om welk cijfer het gaat (One hot encoding). Dit keer gebruiken we hiervoor de `keras` functie `to_categorical`.

In [341]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print(train_labels)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


Maak nu een neuraal netwerk met twee lagen, een eerste laag met 512 neuronen en een output laag met 10 neuronen. Geef bij de eerste laag aan hoeveel inputs er zijn en wat de activatiefunctie is. De output laag heeft een `softmax` activatiefunctie.

In [342]:
model = Sequential()

#voeg lagen toe met model.add()
# Ik gebruik relu omdat het gaat om binaire getallen en het gemakkelijk te trainen is tenopzichte van sigmoid
model.add(Dense(512, input_shape = (28*28,), activation = 'relu')) 
model.add(Dense(10, activation = 'softmax'))


compileer het netwerk.

In [343]:
from keras.optimizers import Adam

model.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_125 (Dense)           (None, 512)               401920    
                                                                 
 dense_126 (Dense)           (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


train het netwerk 5 epochs met een batch_size van 128.

In [344]:
model.fit(train_images, train_labels, epochs =5, batch_size =128)

Epoch 1/5


469/469 [==============================] - 5s 8ms/step - loss: 0.2747 - accuracy: 0.9209
Epoch 2/5
469/469 [==============================] - 4s 8ms/step - loss: 0.1095 - accuracy: 0.9682
Epoch 3/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0715 - accuracy: 0.9790
Epoch 4/5
469/469 [==============================] - 4s 8ms/step - loss: 0.0505 - accuracy: 0.9852
Epoch 5/5
469/469 [==============================] - 4s 7ms/step - loss: 0.0381 - accuracy: 0.9890


Evalueer het netwerk op de test set.

In [345]:
test_loss, test_acc = model.evaluate(train_images, train_labels)
print(test_acc)

1875/1875 [==============================] - 7s 4ms/step - loss: 0.0282 - accuracy: 0.9925
0.9925000071525574


Experimenteer met verschillende netwerken. Varieer het aantal lagen, het aantal neuronen, de activatiefunties en het aantal epochs.

Ik heb hier 7 lagen gebruikt in plaats van 3 gebruikt, mijn verwachting was dat de accuracy een beetje verhoogd wordt, maar dat dit niet drastisch is omdat de accuracy al heel hoog is.

Dit was niet het geval, de accuracy bleef ongeveer het zelfde, in sommige gevallen was het lager en in sommige gevallen hoger dan met 3 lagen.

In [346]:
# 7 lagen ipv 3
model1 = Sequential()

# Ik gebruik relu omdat het gaat om binaire getallen en het gemakkelijk te trainen is tenopzichte van sigmoid
model1.add(Dense(512, input_shape = (28*28,), activation = 'relu')) 
model1.add(Dense(250, activation = 'relu')) 
model1.add(Dense(250, activation = 'relu')) 
model1.add(Dense(250, activation = 'relu')) 
model1.add(Dense(250, activation = 'relu')) 
model1.add(Dense(250, activation = 'relu')) 
model1.add(Dense(10, activation = 'softmax'))

model1.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
model1.summary()
model1.fit(train_images, train_labels, epochs =5, batch_size =128)

test_loss, test_acc = model1.evaluate(train_images, train_labels)
print(test_acc)

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_127 (Dense)           (None, 512)               401920    
                                                                 
 dense_128 (Dense)           (None, 250)               128250    
                                                                 
 dense_129 (Dense)           (None, 250)               62750     
                                                                 
 dense_130 (Dense)           (None, 250)               62750     
                                                                 
 dense_131 (Dense)           (None, 250)               62750     
                                                                 
 dense_132 (Dense)           (None, 250)               62750     
                                                                 
 dense_133 (Dense)           (None, 10)              

Hier test ik het verschil tussen relu en sigmoid, ik had verwacht dat het ongeveer het zelfde resultaat zou leveren, maar het bleek dat sigmoid in dit geval een slechtere accuracy leverde dan relu.

In [347]:
model2 = Sequential()

model2.add(Dense(512, input_shape = (28*28,), activation = 'sigmoid')) 
model2.add(Dense(250, activation = 'sigmoid')) 
model2.add(Dense(10, activation = 'softmax'))

model2.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
model2.summary()
model2.fit(train_images, train_labels, epochs =5, batch_size =128)

test_loss, test_acc = model2.evaluate(train_images, train_labels)
print(test_acc)

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_134 (Dense)           (None, 512)               401920    
                                                                 
 dense_135 (Dense)           (None, 250)               128250    
                                                                 
 dense_136 (Dense)           (None, 10)                2510      
                                                                 
Total params: 532680 (2.03 MB)
Trainable params: 532680 (2.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/5
469/469 [==============================] - 5s 9ms/step - loss: 0.5265 - accuracy: 0.8552
Epoch 2/5
469/469 [==============================] - 4s 9ms/step - loss: 0.2143 - accuracy: 0.9361
Epoch 3/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1580 - accuracy: 0.9530
Epoch 4/5
469/469 [==============================] - 4s 9ms/step - loss: 0.1221 - accuracy: 0.9638
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0774 - accuracy: 0.9771
0.9771166443824768


Ik test hier of de hoeveelheid epochs invloed heeft op de accuracy, mijn voorspelling was hoe meer epochs hoe hoger het accuracy zou zijn. In werkelijkheid heeft het weinig invloed op de accuracy.

In [348]:
model3 = Sequential()

model3.add(Dense(512, input_shape = (28*28,), activation = 'relu')) 
model3.add(Dense(250, activation = 'relu')) 
model3.add(Dense(10, activation = 'softmax'))

model3.compile(Adam(lr=0.01),'categorical_crossentropy',metrics=['accuracy'])
model3.summary()
model3.fit(train_images, train_labels, epochs =10, batch_size =128)

test_loss, test_acc = model3.evaluate(train_images, train_labels)
print(test_acc)

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_137 (Dense)           (None, 512)               401920    
                                                                 
 dense_138 (Dense)           (None, 250)               128250    
                                                                 
 dense_139 (Dense)           (None, 10)                2510      
                                                                 
Total params: 532680 (2.03 MB)
Trainable params: 532680 (2.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/10
469/469 [==============================] - 7s 11ms/step - loss: 0.2316 - accuracy: 0.9329
Epoch 2/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0820 - accuracy: 0.9747
Epoch 3/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0526 - accuracy: 0.9837
Epoch 4/10
469/469 [==============================] - 5s 10ms/step - loss: 0.0379 - accuracy: 0.9878
Epoch 5/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0254 - accuracy: 0.9919
Epoch 6/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0230 - accuracy: 0.9926
Epoch 7/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0176 - accuracy: 0.9940
Epoch 8/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0142 - accuracy: 0.9952
Epoch 9/10
469/469 [==============================] - 5s 11ms/step - loss: 0.0144 - accuracy: 0.9952
Epoch 10/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0147 - accur